# 01b: Linear Algebra Refresh

**Week 1, Days 3-4** | Foundations

**Prerequisites**: 01a (tensors, broadcasting, indexing)

---

## Learning Objectives

By the end of this notebook, you will be able to:

- [ ] Visualize vectors as arrows in space
- [ ] Understand matrices as transformations (rotation, scaling, shearing)
- [ ] Compute and interpret eigenvalues and eigenvectors
- [ ] Use SVD for dimensionality reduction
- [ ] Connect these concepts to machine learning applications

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
import torch

# Config
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# For reproducibility
np.random.seed(42)

---

# Part 1: Vectors

## Layer 1: Intuition — Arrows in space

Forget lists of numbers for a moment. A **vector** is an arrow:

- It has a **direction** (where it points)
- It has a **magnitude** (how long it is)
- It does NOT have a fixed position (you can slide it around)

Think of a vector as a **displacement**: "move 3 steps east and 4 steps north." It doesn't matter where you start—the displacement is the same.

**Physical analogies**:
- **Velocity**: speed + direction (a car going 60 mph north)
- **Force**: strength + direction (pushing a box with 10N to the right)
- **Wind**: speed + direction

In machine learning, we extend this to high dimensions:
- A word embedding is a vector in 300-dimensional space
- An image is a vector in (height × width × channels)-dimensional space
- Each dimension captures some feature

## Layer 2: Code + Visualization

In [ ]:
# Helper function to plot vectors
def plot_vectors(vectors, colors=None, labels=None, title="Vectors", ax=None):
    """
    Plot 2D vectors as arrows from the origin.
    
    Args:
        vectors: List of 2D vectors (numpy arrays)
        colors: Optional list of colors
        labels: Optional list of labels
        title: Plot title
        ax: Optional matplotlib axis
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    if colors is None:
        colors = plt.cm.tab10(np.linspace(0, 1, len(vectors)))
    
    # Find plot bounds
    all_coords = np.array(vectors)
    max_val = np.abs(all_coords).max() * 1.3
    
    for i, v in enumerate(vectors):
        color = colors[i] if isinstance(colors, (list, np.ndarray)) else colors
        label = labels[i] if labels else None
        ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1,
                  color=color, width=0.02, label=label)
    
    ax.set_xlim(-max_val, max_val)
    ax.set_ylim(-max_val, max_val)
    ax.set_aspect('equal')
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.grid(True, alpha=0.3)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(title)
    if labels:
        ax.legend()
    
    return ax

In [ ]:
# Visualize basic vector operations

v = np.array([3, 2])
w = np.array([1, 4])

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Two vectors
plot_vectors([v, w], colors=['blue', 'red'], labels=['v = [3,2]', 'w = [1,4]'],
             title='Two vectors', ax=axes[0])

# Plot 2: Vector addition (parallelogram rule)
plot_vectors([v, w, v + w], colors=['blue', 'red', 'green'],
             labels=['v', 'w', 'v + w'], title='Vector Addition', ax=axes[1])
# Show the parallelogram
axes[1].plot([v[0], v[0]+w[0]], [v[1], v[1]+w[1]], 'r--', alpha=0.5)
axes[1].plot([w[0], v[0]+w[0]], [w[1], v[1]+w[1]], 'b--', alpha=0.5)

# Plot 3: Scalar multiplication
plot_vectors([v, 2*v, -0.5*v], colors=['blue', 'green', 'orange'],
             labels=['v', '2v', '-0.5v'], title='Scalar Multiplication', ax=axes[2])

plt.tight_layout()
plt.show()

In [ ]:
# Vector magnitude and direction

v = np.array([3, 4])

# Magnitude (length): ||v|| = sqrt(v·v)
magnitude = np.linalg.norm(v)
print(f"v = {v}")
print(f"||v|| = sqrt(3² + 4²) = {magnitude}")

# Unit vector (direction only): v / ||v||
unit = v / magnitude
print(f"Unit vector: {unit}")
print(f"Unit vector magnitude: {np.linalg.norm(unit)}")

# Visualize
fig, ax = plt.subplots(figsize=(8, 6))
plot_vectors([v, unit*2], colors=['blue', 'red'],
             labels=[f'v (||v||={magnitude})', 'unit vector (scaled 2x for visibility)'],
             title='Vector and its unit vector', ax=ax)
plt.show()

## Layer 3: CS Speak

### Vector Operations and Complexity

| Operation | Formula | Complexity | NumPy |
|-----------|---------|------------|-------|
| Addition | $\mathbf{u} + \mathbf{v}$ | O(n) | `u + v` |
| Scalar mult | $c\mathbf{v}$ | O(n) | `c * v` |
| Dot product | $\mathbf{u} \cdot \mathbf{v}$ | O(n) | `np.dot(u, v)` or `u @ v` |
| Magnitude | $\|\mathbf{v}\|$ | O(n) | `np.linalg.norm(v)` |
| Normalize | $\mathbf{v} / \|\mathbf{v}\|$ | O(n) | `v / np.linalg.norm(v)` |

### The Dot Product: Similarity Measure

The dot product $\mathbf{u} \cdot \mathbf{v}$ tells you:
- **Positive**: vectors point in similar directions
- **Zero**: vectors are perpendicular (orthogonal)
- **Negative**: vectors point in opposite directions

In ML, **cosine similarity** is just the dot product of unit vectors:

$$\cos(\theta) = \frac{\mathbf{u} \cdot \mathbf{v}}{\|\mathbf{u}\| \|\mathbf{v}\|}$$

This is how we measure "how similar are these two embeddings?"

In [ ]:
# Dot product and angle between vectors

def angle_between(u, v):
    """Compute angle between two vectors in degrees."""
    cos_angle = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    # Clamp to [-1, 1] to handle numerical errors
    cos_angle = np.clip(cos_angle, -1, 1)
    return np.degrees(np.arccos(cos_angle))

# Examples
pairs = [
    (np.array([1, 0]), np.array([1, 0])),     # Same direction
    (np.array([1, 0]), np.array([0, 1])),     # Perpendicular
    (np.array([1, 0]), np.array([-1, 0])),    # Opposite
    (np.array([1, 1]), np.array([1, 0])),     # 45 degrees
]

print("Dot product and angles:\n")
for u, v in pairs:
    dot = np.dot(u, v)
    angle = angle_between(u, v)
    print(f"u={u}, v={v}")
    print(f"  u·v = {dot:.2f}, angle = {angle:.1f}°\n")

## Layer 4: Mathematical Formalism

### Definitions

A **vector** $\mathbf{v} \in \mathbb{R}^n$ is an ordered tuple of $n$ real numbers:

$$\mathbf{v} = \begin{bmatrix} v_1 \\ v_2 \\ \vdots \\ v_n \end{bmatrix}$$

The **dot product** (inner product) of $\mathbf{u}, \mathbf{v} \in \mathbb{R}^n$:

$$\mathbf{u} \cdot \mathbf{v} = \sum_{i=1}^{n} u_i v_i = \mathbf{u}^T \mathbf{v}$$

The **Euclidean norm** (length, magnitude):

$$\|\mathbf{v}\| = \sqrt{\mathbf{v} \cdot \mathbf{v}} = \sqrt{\sum_{i=1}^{n} v_i^2}$$

### Key Properties

**Cauchy-Schwarz inequality**:
$$|\mathbf{u} \cdot \mathbf{v}| \leq \|\mathbf{u}\| \|\mathbf{v}\|$$

with equality iff $\mathbf{u} = c\mathbf{v}$ for some scalar $c$.

**Geometric interpretation of dot product**:
$$\mathbf{u} \cdot \mathbf{v} = \|\mathbf{u}\| \|\mathbf{v}\| \cos(\theta)$$

where $\theta$ is the angle between the vectors.

---

# Part 2: Matrices as Transformations

## Layer 1: Intuition — Shape-shifting machines

A matrix is not just a table of numbers. Think of it as a **transformation machine**:

- You feed in a vector
- It spits out a transformed vector

What kinds of transformations? ALL linear ones:

| Transformation | Visual | What it does |
|----------------|--------|-------------|
| **Rotation** | 🔄 | Spins space around the origin |
| **Scaling** | ↔️↕️ | Stretches/compresses along axes |
| **Reflection** | 🪞 | Flips across a line |
| **Shearing** | 📐 | Tilts space (like italicizing text) |
| **Projection** | 📽️ | Flattens onto a subspace |

**Key insight**: Matrix multiplication = applying a transformation. $\mathbf{A}\mathbf{v}$ means "apply transformation $\mathbf{A}$ to vector $\mathbf{v}$."

**Physical analogy**: Think of a fun-house mirror. Different mirrors (matrices) distort your reflection (vectors) in different ways. Some stretch you tall, some squish you wide, some twist you around.

## Layer 2: Code + Visualization

In [ ]:
# Helper: visualize how a matrix transforms the unit circle

def plot_transformation(A, title="Transformation", ax=None):
    """
    Show how matrix A transforms the unit circle and basis vectors.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))
    
    # Unit circle
    theta = np.linspace(0, 2*np.pi, 100)
    circle = np.array([np.cos(theta), np.sin(theta)])
    
    # Transformed circle
    transformed = A @ circle
    
    # Plot
    ax.plot(circle[0], circle[1], 'b-', linewidth=2, label='Unit circle', alpha=0.5)
    ax.plot(transformed[0], transformed[1], 'r-', linewidth=2, label='Transformed')
    
    # Basis vectors and their transforms
    e1, e2 = np.array([1, 0]), np.array([0, 1])
    Ae1, Ae2 = A @ e1, A @ e2
    
    ax.quiver(0, 0, e1[0], e1[1], angles='xy', scale_units='xy', scale=1,
              color='blue', width=0.02, alpha=0.7, label='e₁ = [1,0]')
    ax.quiver(0, 0, e2[0], e2[1], angles='xy', scale_units='xy', scale=1,
              color='blue', width=0.02, alpha=0.7, label='e₂ = [0,1]')
    ax.quiver(0, 0, Ae1[0], Ae1[1], angles='xy', scale_units='xy', scale=1,
              color='red', width=0.02, label=f'Ae₁ = [{Ae1[0]:.1f},{Ae1[1]:.1f}]')
    ax.quiver(0, 0, Ae2[0], Ae2[1], angles='xy', scale_units='xy', scale=1,
              color='darkred', width=0.02, label=f'Ae₂ = [{Ae2[0]:.1f},{Ae2[1]:.1f}]')
    
    # Formatting
    all_coords = np.concatenate([circle, transformed], axis=1)
    max_val = np.abs(all_coords).max() * 1.2
    ax.set_xlim(-max_val, max_val)
    ax.set_ylim(-max_val, max_val)
    ax.set_aspect('equal')
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.grid(True, alpha=0.3)
    ax.set_title(f"{title}\nA = {A.tolist()}")
    ax.legend(loc='upper left', fontsize=8)
    
    return ax

In [ ]:
# Common transformations

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Identity
I = np.array([[1, 0], [0, 1]])
plot_transformation(I, "Identity", axes[0, 0])

# Scaling
S = np.array([[2, 0], [0, 0.5]])
plot_transformation(S, "Scaling (2x horizontal, 0.5x vertical)", axes[0, 1])

# Rotation (45 degrees)
theta = np.pi / 4
R = np.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta), np.cos(theta)]])
plot_transformation(R, "Rotation (45°)", axes[0, 2])

# Shear
H = np.array([[1, 0.5], [0, 1]])
plot_transformation(H, "Horizontal Shear", axes[1, 0])

# Reflection (across x-axis)
F = np.array([[1, 0], [0, -1]])
plot_transformation(F, "Reflection (across x-axis)", axes[1, 1])

# Projection (onto x-axis)
P = np.array([[1, 0], [0, 0]])
plot_transformation(P, "Projection (onto x-axis)", axes[1, 2])

plt.tight_layout()
plt.show()

In [ ]:
# Matrix multiplication = composition of transformations

# First rotate 45°, then scale
theta = np.pi / 4
R = np.array([[np.cos(theta), -np.sin(theta)],
              [np.sin(theta), np.cos(theta)]])
S = np.array([[2, 0], [0, 0.5]])

# Composition: apply R first, then S
# Note: S @ R means "first R, then S" (right-to-left)
composed = S @ R

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

plot_transformation(R, "Step 1: Rotate 45°", axes[0])
plot_transformation(S, "Step 2: Scale", axes[1])
plot_transformation(composed, "Combined: Scale(Rotate(·))", axes[2])

plt.tight_layout()
plt.show()

print("Note: Matrix multiplication is NOT commutative!")
print(f"S @ R =\n{S @ R}")
print(f"R @ S =\n{R @ S}")
print(f"Equal? {np.allclose(S @ R, R @ S)}")

## Layer 3: CS Speak

### Matrix Multiplication

For $\mathbf{A} \in \mathbb{R}^{m \times k}$ and $\mathbf{B} \in \mathbb{R}^{k \times n}$:

$$[\mathbf{A}\mathbf{B}]_{ij} = \sum_{l=1}^{k} A_{il} B_{lj}$$

**Complexity**: $O(m \cdot k \cdot n)$ — this is why large neural networks are expensive!

### Matrix Properties

| Property | Definition | Meaning |
|----------|------------|---------|
| **Symmetric** | $\mathbf{A} = \mathbf{A}^T$ | Same transformation and its "reverse" |
| **Orthogonal** | $\mathbf{A}^T \mathbf{A} = \mathbf{I}$ | Preserves lengths and angles (rotation/reflection) |
| **Positive definite** | $\mathbf{x}^T \mathbf{A} \mathbf{x} > 0$ | All eigenvalues positive |
| **Invertible** | $\det(\mathbf{A}) \neq 0$ | Transformation can be undone |

### Key NumPy/PyTorch Functions

```python
A @ B           # Matrix multiplication
A.T             # Transpose
np.linalg.inv(A)   # Inverse
np.linalg.det(A)   # Determinant
np.trace(A)     # Trace (sum of diagonal)
```

## Layer 4: Mathematical Formalism

### Linear Transformations

A function $T: \mathbb{R}^n \to \mathbb{R}^m$ is **linear** if:

1. $T(\mathbf{u} + \mathbf{v}) = T(\mathbf{u}) + T(\mathbf{v})$ (additivity)
2. $T(c\mathbf{v}) = cT(\mathbf{v})$ (homogeneity)

**Theorem**: Every linear transformation can be represented by a matrix.

If $\{\mathbf{e}_1, \ldots, \mathbf{e}_n\}$ is the standard basis, then:

$$\mathbf{A} = [T(\mathbf{e}_1) | T(\mathbf{e}_2) | \cdots | T(\mathbf{e}_n)]$$

That is, **the columns of $\mathbf{A}$ are where the basis vectors land**.

### The Determinant

The determinant $\det(\mathbf{A})$ measures how much the transformation scales area (2D) or volume (3D):

$$|\det(\mathbf{A})| = \frac{\text{area of transformed unit square}}{\text{area of unit square}}$$

- $|\det| > 1$: expansion
- $|\det| < 1$: compression
- $|\det| = 0$: collapses to lower dimension (not invertible)
- $\det < 0$: includes reflection (orientation flip)

In [ ]:
# Determinant = area scaling factor

matrices = [
    (np.array([[2, 0], [0, 2]]), "Uniform scaling (2x)"),
    (np.array([[2, 0], [0, 0.5]]), "Non-uniform scaling"),
    (np.array([[1, 1], [0, 1]]), "Shear"),
    (np.array([[0, 1], [1, 0]]), "Reflection"),
    (np.array([[1, 2], [2, 4]]), "Singular (collapses)"),
]

print("Matrix determinants and their meanings:\n")
for A, name in matrices:
    det = np.linalg.det(A)
    print(f"{name}:")
    print(f"  A = {A.tolist()}")
    print(f"  det(A) = {det:.2f}")
    if det == 0:
        print(f"  → Singular! Collapses space.")
    elif det < 0:
        print(f"  → Includes reflection, scales area by {abs(det):.2f}x")
    else:
        print(f"  → Scales area by {det:.2f}x")
    print()

---

# Part 3: Eigenvalues and Eigenvectors

## Layer 1: Intuition — Special directions that only stretch

Most vectors get rotated AND stretched when you apply a matrix. But some special vectors **only get stretched** (or flipped) — they keep pointing in the same direction.

These are **eigenvectors**. The amount they stretch by is the **eigenvalue**.

**Physical analogy**: Imagine pushing a playground merry-go-round:
- Most points move in circles (rotation + radial motion)
- But the center point stays put (eigenvector with eigenvalue 0)
- And points along the axis of rotation only move up/down (eigenvector along the axis)

**Why do we care?**
- Eigenvectors reveal the "natural axes" of a transformation
- PCA finds eigenvectors of the covariance matrix → principal directions of variation
- In dynamical systems, eigenvalues determine stability
- Google's PageRank is based on finding the principal eigenvector

## Layer 2: Code + Visualization

In [ ]:
# Simple eigenvector example

A = np.array([[3, 1],
              [0, 2]])

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(A)

print(f"Matrix A:\n{A}\n")
print(f"Eigenvalues: {eigenvalues}")
print(f"Eigenvectors (as columns):\n{eigenvectors}\n")

# Verify: A @ v = λ @ v
for i, (lam, v) in enumerate(zip(eigenvalues, eigenvectors.T)):
    Av = A @ v
    lam_v = lam * v
    print(f"Eigenvector {i+1}: v = {v}")
    print(f"  A @ v  = {Av}")
    print(f"  λ * v  = {lam_v}")
    print(f"  Equal? {np.allclose(Av, lam_v)}\n")

In [ ]:
# Visualize eigenvectors: they only get scaled, not rotated

A = np.array([[2, 1],
              [1, 2]])

eigenvalues, eigenvectors = np.linalg.eig(A)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Show transformation of random vectors
ax = axes[0]
n_vectors = 8
angles = np.linspace(0, 2*np.pi, n_vectors, endpoint=False)
vectors = np.array([[np.cos(a), np.sin(a)] for a in angles])

for v in vectors:
    Av = A @ v
    ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1,
              color='blue', alpha=0.5, width=0.01)
    ax.quiver(0, 0, Av[0], Av[1], angles='xy', scale_units='xy', scale=1,
              color='red', alpha=0.5, width=0.01)

# Highlight eigenvectors
for i, (lam, v) in enumerate(zip(eigenvalues, eigenvectors.T)):
    Av = A @ v
    ax.quiver(0, 0, v[0], v[1], angles='xy', scale_units='xy', scale=1,
              color='blue', width=0.02, label=f'v{i+1} (before)' if i==0 else None)
    ax.quiver(0, 0, Av[0], Av[1], angles='xy', scale_units='xy', scale=1,
              color='green', width=0.02, label=f'Av{i+1} = {lam:.1f}·v{i+1}' if i==0 else None)

ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.grid(True, alpha=0.3)
ax.set_title('Blue: original vectors, Red: transformed\nGreen: eigenvectors (only scaled!)')
ax.legend()

# Right: Show eigenvalues on spectrum
ax = axes[1]
ax.axhline(y=0, color='k', linewidth=2)
for i, lam in enumerate(eigenvalues):
    ax.scatter(lam, 0, s=200, c=f'C{i}', zorder=5)
    ax.annotate(f'λ{i+1} = {lam:.2f}', (lam, 0.1), ha='center', fontsize=12)

ax.set_xlim(-1, 4)
ax.set_ylim(-0.5, 0.5)
ax.set_xlabel('Eigenvalue')
ax.set_title(f'Eigenvalue spectrum of A\nA = {A.tolist()}')
ax.set_yticks([])

plt.tight_layout()
plt.show()

In [ ]:
# What eigenvalues tell us about repeated application

# Matrix with |λ| < 1: repeated application shrinks
A_shrink = np.array([[0.8, 0.1],
                      [0.1, 0.8]])

# Matrix with |λ| > 1: repeated application grows
A_grow = np.array([[1.2, 0.1],
                   [0.1, 1.2]])

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

for ax, A, name in [(axes[0], A_shrink, 'Contracting'), 
                     (axes[1], A_grow, 'Expanding')]:
    eigenvalues, _ = np.linalg.eig(A)
    
    # Start with a random point
    v = np.array([1.0, 0.5])
    trajectory = [v.copy()]
    
    # Apply A repeatedly
    for _ in range(20):
        v = A @ v
        trajectory.append(v.copy())
    
    trajectory = np.array(trajectory)
    
    ax.plot(trajectory[:, 0], trajectory[:, 1], 'b.-', markersize=8)
    ax.scatter(trajectory[0, 0], trajectory[0, 1], c='green', s=100, zorder=5, label='Start')
    ax.scatter(trajectory[-1, 0], trajectory[-1, 1], c='red', s=100, zorder=5, label='End')
    
    ax.axhline(y=0, color='k', linewidth=0.5)
    ax.axvline(x=0, color='k', linewidth=0.5)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_title(f'{name}: eigenvalues = {eigenvalues.round(2)}\n'
                 f'Max |λ| = {np.abs(eigenvalues).max():.2f}')

plt.tight_layout()
plt.show()

print("Key insight: If max|λ| < 1, trajectories converge to 0")
print("            If max|λ| > 1, trajectories diverge to infinity")
print("            If max|λ| = 1, trajectories stay bounded")

## Layer 3: CS Speak

### Computing Eigenvalues

```python
# Full eigendecomposition
eigenvalues, eigenvectors = np.linalg.eig(A)

# For symmetric matrices (more stable)
eigenvalues, eigenvectors = np.linalg.eigh(A)

# Just eigenvalues
eigenvalues = np.linalg.eigvals(A)
```

### Eigendecomposition

If $\mathbf{A}$ has $n$ linearly independent eigenvectors:

$$\mathbf{A} = \mathbf{V} \mathbf{\Lambda} \mathbf{V}^{-1}$$

where:
- $\mathbf{V}$ has eigenvectors as columns
- $\mathbf{\Lambda}$ is diagonal with eigenvalues

**Power**: This makes $\mathbf{A}^n$ easy to compute:
$$\mathbf{A}^n = \mathbf{V} \mathbf{\Lambda}^n \mathbf{V}^{-1}$$

### Applications in ML

| Application | What we compute |
|-------------|----------------|
| **PCA** | Eigenvectors of covariance matrix |
| **Spectral clustering** | Eigenvectors of Laplacian |
| **PageRank** | Principal eigenvector of link matrix |
| **Stability analysis** | Eigenvalues of Jacobian |

## Layer 4: Mathematical Formalism

### Definition

For matrix $\mathbf{A} \in \mathbb{R}^{n \times n}$, scalar $\lambda$ is an **eigenvalue** and non-zero vector $\mathbf{v}$ is an **eigenvector** if:

$$\mathbf{A}\mathbf{v} = \lambda \mathbf{v}$$

Equivalently:

$$(\mathbf{A} - \lambda \mathbf{I})\mathbf{v} = \mathbf{0}$$

For non-trivial solutions, we need:

$$\det(\mathbf{A} - \lambda \mathbf{I}) = 0$$

This is the **characteristic polynomial**, a degree-$n$ polynomial in $\lambda$.

### Properties

**Trace**: $\text{tr}(\mathbf{A}) = \sum_i \lambda_i$ (sum of eigenvalues = sum of diagonal)

**Determinant**: $\det(\mathbf{A}) = \prod_i \lambda_i$ (product of eigenvalues)

**Symmetric matrices**: Real eigenvalues, orthogonal eigenvectors

**Spectral theorem**: For symmetric $\mathbf{A}$:
$$\mathbf{A} = \mathbf{Q} \mathbf{\Lambda} \mathbf{Q}^T$$

where $\mathbf{Q}$ is orthogonal ($\mathbf{Q}^T\mathbf{Q} = \mathbf{I}$).

---

# Part 4: Singular Value Decomposition (SVD)

## Layer 1: Intuition — The ultimate matrix X-ray

Eigendecomposition only works for square matrices. **SVD works for ANY matrix** and reveals its fundamental structure.

Think of SVD as taking an X-ray of a matrix to see its "bones":

1. **What directions in input space matter most?** (right singular vectors)
2. **How much does each direction get stretched?** (singular values)
3. **Where do they end up in output space?** (left singular vectors)

**Physical analogy**: Imagine clay going through a pasta machine:
- The machine can rotate the clay (V)
- Stretch it in certain directions (Σ)
- Then rotate the output (U)

SVD finds the simplest decomposition: rotate → stretch → rotate.

**Killer application**: **Low-rank approximation**. Keep only the top $k$ singular values to compress data while preserving the most important structure.

## Layer 2: Code + Visualization

In [ ]:
# SVD basics

A = np.array([[3, 2, 2],
              [2, 3, -2]])

U, S, Vt = np.linalg.svd(A)

print(f"Original A ({A.shape[0]}×{A.shape[1]}):")
print(A)
print(f"\nU ({U.shape[0]}×{U.shape[1]}):  Left singular vectors")
print(U)
print(f"\nS ({S.shape}):  Singular values")
print(S)
print(f"\nVᵀ ({Vt.shape[0]}×{Vt.shape[1]}):  Right singular vectors (transposed)")
print(Vt)

# Reconstruct
S_matrix = np.zeros_like(A, dtype=float)
np.fill_diagonal(S_matrix, S)
A_reconstructed = U @ S_matrix @ Vt
print(f"\nReconstruction U @ Σ @ Vᵀ:")
print(A_reconstructed)
print(f"\nReconstruction error: {np.linalg.norm(A - A_reconstructed):.2e}")

In [ ]:
# Visualize SVD as rotation-stretch-rotation

A = np.array([[2, 1],
              [1, 2]])

U, S, Vt = np.linalg.svd(A)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Unit circle
theta = np.linspace(0, 2*np.pi, 100)
circle = np.array([np.cos(theta), np.sin(theta)])

# Step 1: Original
ax = axes[0]
ax.plot(circle[0], circle[1], 'b-', linewidth=2)
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.set_title('1. Unit circle\n(input)')
ax.grid(True, alpha=0.3)

# Step 2: After Vᵀ (rotation)
after_Vt = Vt @ circle
ax = axes[1]
ax.plot(after_Vt[0], after_Vt[1], 'g-', linewidth=2)
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.set_title('2. After Vᵀ\n(rotate in input space)')
ax.grid(True, alpha=0.3)

# Step 3: After Σ (stretch)
after_S = np.diag(S) @ after_Vt
ax = axes[2]
ax.plot(after_S[0], after_S[1], 'm-', linewidth=2)
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.set_title(f'3. After Σ\n(stretch by σ₁={S[0]:.2f}, σ₂={S[1]:.2f})')
ax.grid(True, alpha=0.3)

# Step 4: After U (rotation)
after_U = U @ after_S
ax = axes[3]
ax.plot(after_U[0], after_U[1], 'r-', linewidth=2)
ax.set_xlim(-4, 4)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.axhline(y=0, color='k', linewidth=0.5)
ax.axvline(x=0, color='k', linewidth=0.5)
ax.set_title('4. After U\n(rotate in output space = final)')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"A = U @ Σ @ Vᵀ")
print(f"Any matrix = (rotation) @ (stretch) @ (rotation)")

In [ ]:
# Low-rank approximation: compress a matrix

# Create a "signal + noise" matrix
np.random.seed(42)
m, n = 100, 50
true_rank = 5

# Low-rank signal
signal = np.random.randn(m, true_rank) @ np.random.randn(true_rank, n)
# Add noise
noise = 0.5 * np.random.randn(m, n)
A = signal + noise

print(f"Matrix A: {A.shape}")
print(f"True underlying rank: {true_rank}")
print(f"With noise, numerical rank: {np.linalg.matrix_rank(A)}")

# SVD
U, S, Vt = np.linalg.svd(A, full_matrices=False)

# Plot singular values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.plot(S, 'bo-', markersize=4)
ax.axhline(y=S[true_rank-1], color='r', linestyle='--', label=f'σ_{true_rank} (signal cutoff)')
ax.set_xlabel('Index')
ax.set_ylabel('Singular value')
ax.set_title('Singular value spectrum\n(signal lives in top values)')
ax.legend()
ax.grid(True, alpha=0.3)

# Reconstruction error vs rank
ax = axes[1]
ranks = range(1, 20)
errors = []
for k in ranks:
    A_k = U[:, :k] @ np.diag(S[:k]) @ Vt[:k, :]
    error = np.linalg.norm(A - A_k) / np.linalg.norm(A)
    errors.append(error)

ax.plot(ranks, errors, 'go-', markersize=6)
ax.axvline(x=true_rank, color='r', linestyle='--', label=f'True rank ({true_rank})')
ax.set_xlabel('Rank k')
ax.set_ylabel('Relative reconstruction error')
ax.set_title('Low-rank approximation error\n(elbow at true rank)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Energy captured
total_energy = np.sum(S**2)
cumulative_energy = np.cumsum(S**2) / total_energy
print(f"\nEnergy captured by top k singular values:")
for k in [1, 5, 10, 20]:
    print(f"  k={k}: {cumulative_energy[k-1]*100:.1f}%")

## Layer 3: CS Speak

### SVD Decomposition

For any matrix $\mathbf{A} \in \mathbb{R}^{m \times n}$:

```python
U, S, Vt = np.linalg.svd(A, full_matrices=False)
# U: (m, k) left singular vectors
# S: (k,) singular values (sorted descending)
# Vt: (k, n) right singular vectors (transposed)
# where k = min(m, n)
```

### Low-Rank Approximation

The **rank-k approximation** keeps only the top k components:

```python
A_k = U[:, :k] @ np.diag(S[:k]) @ Vt[:k, :]
```

**Eckart-Young theorem**: This is the BEST rank-k approximation (minimizes Frobenius norm error).

### Applications

| Application | How SVD is used |
|-------------|----------------|
| **Image compression** | Keep top k singular values |
| **Recommender systems** | Low-rank matrix factorization |
| **LSA (text)** | Dimensionality reduction on term-document matrix |
| **Pseudoinverse** | $\mathbf{A}^+ = \mathbf{V}\mathbf{\Sigma}^+\mathbf{U}^T$ |
| **PCA** | SVD of centered data matrix |

### Complexity

Full SVD: $O(\min(mn^2, m^2n))$ — expensive for large matrices!

Truncated SVD (top k): $O(mnk)$ — use `scipy.sparse.linalg.svds` for large sparse matrices.

## Layer 4: Mathematical Formalism

### Theorem (SVD)

For any $\mathbf{A} \in \mathbb{R}^{m \times n}$ with rank $r$, there exist:
- Orthogonal $\mathbf{U} \in \mathbb{R}^{m \times m}$
- Orthogonal $\mathbf{V} \in \mathbb{R}^{n \times n}$
- Diagonal $\mathbf{\Sigma} \in \mathbb{R}^{m \times n}$ with $\sigma_1 \geq \sigma_2 \geq \cdots \geq \sigma_r > 0$

such that:

$$\mathbf{A} = \mathbf{U}\mathbf{\Sigma}\mathbf{V}^T = \sum_{i=1}^{r} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$

### Relationship to Eigendecomposition

- Left singular vectors $\mathbf{u}_i$ are eigenvectors of $\mathbf{A}\mathbf{A}^T$
- Right singular vectors $\mathbf{v}_i$ are eigenvectors of $\mathbf{A}^T\mathbf{A}$
- Singular values: $\sigma_i = \sqrt{\lambda_i}$ where $\lambda_i$ are eigenvalues of $\mathbf{A}^T\mathbf{A}$

### Eckart-Young-Mirsky Theorem

The rank-$k$ truncated SVD:

$$\mathbf{A}_k = \sum_{i=1}^{k} \sigma_i \mathbf{u}_i \mathbf{v}_i^T$$

minimizes $\|\mathbf{A} - \mathbf{B}\|_F$ over all rank-$k$ matrices $\mathbf{B}$.

The error is:

$$\|\mathbf{A} - \mathbf{A}_k\|_F = \sqrt{\sigma_{k+1}^2 + \cdots + \sigma_r^2}$$

---

# Exercises

Now it's your turn.

## Exercise 1: Transformation Visualization

Create a 2×2 matrix that:
1. First stretches by 2x horizontally
2. Then rotates by 30 degrees

Plot the unit circle and its transformation. Verify your matrix is correct.

In [ ]:
# Exercise 1: Transformation Visualization

def create_stretch_rotate_matrix(stretch_x, angle_degrees):
    """
    Create a matrix that first stretches horizontally, then rotates.
    
    Args:
        stretch_x: Horizontal stretch factor
        angle_degrees: Rotation angle in degrees
    
    Returns:
        2x2 transformation matrix
    """
    # TODO: Create stretch matrix [[stretch_x, 0], [0, 1]]
    # TODO: Create rotation matrix
    # TODO: Compose them (remember: right-to-left order!)
    pass

# Test
A = create_stretch_rotate_matrix(2, 30)
print(f"Transformation matrix:\n{A}")

# Visualize
plot_transformation(A, "Stretch 2x, then rotate 30°")
plt.show()

## Exercise 2: Eigenvalue Intuition

Find a 2×2 matrix where BOTH eigenvalues are negative. What happens to vectors under repeated application of this matrix? Visualize it.

In [ ]:
# Exercise 2: Eigenvalue Intuition

def create_negative_eigenvalue_matrix():
    """
    Create a 2x2 matrix with both eigenvalues negative.
    
    Hint: A diagonal matrix [[a, 0], [0, b]] has eigenvalues a and b.
    Or use the fact that reflection has eigenvalue -1.
    
    Returns:
        2x2 numpy array with both eigenvalues < 0
    """
    # TODO: Create your matrix
    pass

# Test
A = create_negative_eigenvalue_matrix()
eigenvalues = np.linalg.eigvals(A)
print(f"Matrix A:\n{A}")
print(f"Eigenvalues: {eigenvalues}")
print(f"Both negative? {all(eigenvalues < 0)}")

# TODO: Visualize trajectory under repeated application
# What pattern do you observe?

## Exercise 3: SVD Experiment

Take a 100×50 matrix of random numbers. Use SVD to find the rank-10 approximation. 

Questions:
1. How much "energy" (sum of squared singular values) is preserved?
2. What is the compression ratio (original size vs. stored components)?

In [ ]:
# Exercise 3: SVD Experiment

def svd_analysis(m, n, k):
    """
    Analyze rank-k SVD approximation of a random m×n matrix.
    
    Args:
        m, n: Matrix dimensions
        k: Target rank
    
    Returns:
        Dict with analysis results
    """
    # TODO: Create random matrix
    # TODO: Compute SVD
    # TODO: Compute rank-k approximation
    # TODO: Calculate:
    #   - Energy preserved (sum of top k squared singular values / total)
    #   - Reconstruction error
    #   - Compression ratio (m*n original vs. m*k + k + k*n for SVD components)
    pass

# Test
results = svd_analysis(100, 50, 10)
print(f"Results for 100×50 matrix, rank-10 approximation:")
print(f"  Energy preserved: {results['energy_preserved']*100:.1f}%")
print(f"  Relative error: {results['relative_error']:.3f}")
print(f"  Compression ratio: {results['compression_ratio']:.2f}x")

---

# Why This Matters

Linear algebra is the language of machine learning:

| Concept | ML Application |
|---------|---------------|
| **Vectors** | Embeddings, feature vectors, gradients |
| **Dot product** | Similarity, attention scores |
| **Matrix multiplication** | Layer transformations, attention |
| **Eigenvalues** | PCA, stability analysis, spectral methods |
| **SVD** | Dimensionality reduction, compression, recommendations |

In Week 2, we'll see embeddings as vectors in high-dimensional space. The intuition from 2D carries over: similar meanings = similar directions.

In Weeks 5-6, eigenvalues will determine whether dynamical systems converge or diverge—critical for understanding optimization.

---

# Resources

**Essential** (watch these!):
- [3Blue1Brown: Essence of Linear Algebra](https://www.youtube.com/playlist?list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab) — Best visual intuition

**Deep Dives**:
- [Gilbert Strang: Linear Algebra (MIT OCW)](https://ocw.mit.edu/courses/mathematics/18-06-linear-algebra-spring-2010/)
- [Immersive Linear Algebra](http://immersivemath.com/ila/index.html) — Interactive textbook

**Wiki**:
- [Glossary](../../wiki/glossary.md) — eigenvalue, eigenvector, SVD, rank

---

# Reflection Questions

Before moving on:

1. **What's the geometric interpretation of eigenvalues?**
   - Think about what happens to eigenvectors under the transformation...

2. **Why is SVD more general than eigendecomposition?**
   - What kinds of matrices can each handle?

3. **How would you use SVD to compress an image?**
   - Think about treating the image as a matrix...

---

# Next Up

In **01c: Calculus Refresh**, we'll cover:
- Derivatives and gradients
- The chain rule (backbone of backpropagation)
- Gradient descent from scratch

This is where all the pieces come together for optimization.

→ [01c: Calculus Refresh](01c-calculus-refresh.ipynb)